# **using tensorflow keras**
**"Empowering Precision and Quality in Welding through Intelligent Defect Detection."**

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
# Set parameters
img_height, img_width = 224, 224
batch_size = 32
dataset_path = '/content/drive/MyDrive/welding dataset/The Welding Defect Dataset'

# Create ImageDataGenerators for training and validation sets
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    dataset_path + '/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    dataset_path + '/valid',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 839 images belonging to 2 classes.
Found 176 images belonging to 2 classes.


In [ ]:
def create_model(input_shape, num_classes):
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    inputs = layers.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

# Define model parameters
input_shape = (img_height, img_width, 3)
num_classes = len(train_generator.class_indices)

model = create_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=3  # Adjust epochs based on your needs
)


Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


27/27 ━━━━━━━━━━━━━━━━━━━━ 245s 9s/step - accuracy: 0.7602 - loss: 0.4318 - val_accuracy: 1.0000 - val_loss: 9.1723e-06
Epoch 2/3
27/27 ━━━━━━━━━━━━━━━━━━━━ 235s 8s/step - accuracy: 1.0000 - loss: 6.3079e-06 - val_accuracy: 1.0000 - val_loss: 2.0266e-06
Epoch 3/3
27/27 ━━━━━━━━━━━━━━━━━━━━ 202s 7s/step - accuracy: 1.0000 - loss: 1.9657e-06 - val_accuracy: 1.0000 - val_loss: 1.1562e-06


In [ ]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')


6/6 ━━━━━━━━━━━━━━━━━━━━ 35s 6s/step - accuracy: 1.0000 - loss: 1.1463e-06
Validation Loss: 1.1561937753867824e-06, Validation Accuracy: 1.0


In [ ]:
# Save the model in Keras format
model.save('/content/drive/My Drive/welding_defect_model.keras')


In [ ]:
# Load the model from Keras format
model = tf.keras.models.load_model('/content/drive/My Drive/welding_defect_model.keras')


In [ ]:
# Load and preprocess a new image for prediction
def prepare_image(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

# Path to the new image
img_path1 = '//content/drive/MyDrive/welding dataset/The Welding Defect Dataset/test/images/295992d6-f617b725-c1_png_jpg.rf.5f645e13f1695df588b0644ca9702c43.jpg'
img_path2='/content/drive/MyDrive/welding dataset/The Welding Defect Dataset/test/images/Good-Welding-images_23_jpeg_jpg.rf.fdf50352e3c124ff6edab3e96257a040.jpg'
img_array1 = prepare_image(img_path1)
img_array2 = prepare_image(img_path2)

# Make a prediction
predictions1 = model.predict(img_array1)
predictions2 = model.predict(img_array2)
predicted_class1 = np.argmax(predictions1, axis=1)
predicted_class2 = np.argmax(predictions2, axis=1)
class_labels = list(train_generator.class_indices.keys())  # Get class labels
print(f'Predicted Class1: {class_labels[predicted_class1[0]]}')
print(f'Predicted Class2: {class_labels[predicted_class2[0]]}')




1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step
Predicted Class1: images
Predicted Class2: images
